<hr style="border:0.2px solid black"> </hr>

<figure>
  <IMG SRC="data/ntnu_logo.png" WIDTH=250 ALIGN="right">
</figure>

**<ins>Course:</ins>** TVM4174 - Hydroinformatics for Smart Water Systems - Spring 2022

# Using Plotly Package for Interactive Visualization
    
*Developed by Bastian Habbel and Leonardo Sigmund*

<hr style="border:0.2px solid black"> </hr>

    


# Introduction

The plotly Python library is an interactive, open-source plotting library that supports over 40 unique chart types covering a wide range of statistical, financial, geographic, scientific, and 3-dimensional use-cases. Built on top of the Plotly JavaScript library (plotly.js), plotly enables Python users to create interactive web-based visualizations that can be displayed in Jupyter notebooks, saved to standalone HTML files, or served as part of pure Python-built web applications using Dash. The plotly Python library is sometimes referred to as "plotly.py" to differentiate it from the JavaScript library.

Full details for the plotly library are given in the [Plotly Documentation](https://plotly.com/python/). This notebook aims to give a basic understanding of the possibilities of this package in the field of water distribution, general data sciences and mapping.

The sublibraries which are used in this notebook are `plotly.express` as `px`'and `plotly.graph_object` as `go`. Eventhough it's not necessary to reload the package for every chapter, we decided to leave it in the code so that the chapters can be used indepedent.

# Previous knowledge

This notebook requires some previous knowledge some packages. In case you have difficulties in understanding the code, we recommend you to take a look on [Mark Bakker's Tutorials](https://mbakker7.github.io/exploratory_computing_with_python/).
- `matplotbib`: We refer to it in some parts since the working process is similar.  
  - Notebook 1: [Basics and Plotting](https://nbviewer.org/github/mbakker7/exploratory_computing_with_python/blob/master/notebook1_basics_plotting/py_exploratory_comp_1_sol.ipynb)
- `numpy`: Used for functions  
  - Notebook 4: [Functions](https://nbviewer.org/github/mbakker7/exploratory_computing_with_python/blob/master/notebook4_functions/py_exploratory_comp_4_sol.ipynb) 
- `pandas`: In general useful for data handling with dataframes
  - Notebook 8: [Pandas and time series](https://nbviewer.org/github/mbakker7/exploratory_computing_with_python/blob/master/notebook8_pandas/py_exploratory_comp_8_sol.ipynb)
- `wntr`: This package should be known from the class, it is used to analyze water distribution networks. There may be a good notebook available on blackboard which explains the necessary basics for this package.
 
  

# Content

**0. [Installation of the package](#0.-Installation-of-the-package)**

**1. [Plotting data series](#1.-Plotting-data-series)**
  - Basic bar plot
  - Customized plot
  - Range slider and selectors
  - Basic line plot
  
**2. [Statistical Analysis: Histogram](#2-Statistical-Analysis:-Histogram)**

**3. [3D plotting](#3.-3D-plots)**
  - 3D scatter plots
  - 3D surface plots
  
**4. [Map plotting](#4.-Map-plotting)**

**5. [Plotting a water distribution network](#5.-Plotting-a-water-distribution-network)**

**6. [Animations](#6.-Animations)**

**7. [Saving figures](#7.-Saving-figures)**


There are some tasks included, which you can use to test your knowledge. The [Solutions to the tasks](#Task-solutions) are at the end of this notebook.

Let's start!

# 0. Installation of the package

Plotly can be installed using `pip`. Once installed on the kernel, this step can be skipped. Be aware, maybe the version is updated...

In [ ]:
pip install plotly==5.7.0 

# 1. Plotting data series

### Import data

First, we need some example data. In this case, we use annual rainfall data for Rotterdam which is stored under `rotterdam_rainfall_2012.txt`. Here, we prepare the data since we start with a raw textfile.

In [ ]:
import pandas as pd

#1. read data
raindailyRotterdam = pd.read_csv('data/rotterdam_rainfall_2012.txt', sep=',', skiprows=9, parse_dates=['YYYYMMDD'],skipinitialspace=True)

#2. convert to mm/d
raindailyRotterdam.iloc[:,2] = raindailyRotterdam.iloc[:,2] * 0.1

#3. replace false values
raindailyRotterdam.loc[raindailyRotterdam.RH<0.0, 'RH'] = 0.0

### Basic bar plot

The library `express`, usually imported as `xp`, is build on basis of the library `graph_objects`, but makes the code more simple.
Use Plotly Express to show the data in an interactive graph:

In [ ]:
import plotly.express as px

fig = px.bar(raindailyRotterdam, x='YYYYMMDD', y='RH')
fig.show()

You can already zoom in the graph and see more details compared to other methods. Give it a try with your mouse! There is a menu on the top-right corner which e.g. allows you to come back to the initial state.

### Customized plot

We can store and costumize the figure, similar to the procedure used in `matplotlib`:

- `ticklabelmode` indicates where the labels are positioned.
- `dtick` defines the inferval of labels.

In [ ]:
#store the plot in a variable
fig = px.bar(raindailyRotterdam, x='YYYYMMDD', y='RH')

#add a title. The content of the following line could also be part of the brackets above:
fig.update_layout(title = 'Rainfall distribution Rotterdamm')

#costumize the axis
fig.update_xaxes(title='2012', ticklabelmode="period")
fig.update_yaxes(title='Precipitation [mm/day]', dtick='2')

#show the graph
fig.show()

### Rangeslider and selectors

As you may have noticed, it's sometimes hard to zoom into a part of the graph without changing the scaling in y-direction without purpose. Therefore, Plotly has invented additional navigation tools that will be shown in this chapter.

#### Rangeslider
A rangeslider can be added to allow a closer look on the data series by setting the boolean `rangeslider_visible` on true:

In [ ]:
fig = fig.update_xaxes(rangeslider_visible=True)

More information about the rangeslider can be found in the [documentation](https://plotly.com/python/reference/layout/xaxis/#layout-xaxis-rangeslider).

#### Selectors
Additional selectors for pre-defined time-intervals might be useful as well:

In [ ]:
fig = fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=7, label="7 days", step="day", stepmode="todate"),
            dict(count=1, label="1 month", step="month", stepmode="backward"),
            dict(count=6, label="1/2 year", step="month", stepmode="backward"),
            dict(step="all")
        ])
    )
)

It is important to mention that it's necessary to parse the dates before so that plotly knows where to set the timesteps. Same here, in case you have further questions, they are probably answered in the [documentation](https://plotly.com/python/reference/layout/xaxis/#layout-xaxis-rangeselector).


Let's have a look on the result:

In [ ]:
fig.show()

The selectors appear on top of the graph and the slider on the bottom.

Give it a try! You can always come back to the initial view using the navigation in the top-right corner.

### Basic line plot

Some data, for instance yearly trends, are better to show in a line than in a barplot. Plotly express provides interactive line charts for this purpose as well as will be shown in the following. 

Gapminder Foundation is a non-profit venture registered in Stockholm, Sweden, that promotes sustainable global development and achievement of the United Nations Millennium Development Goals by increased use and understanding of statistics and other information about social, economic and environmental development at local, national and global levels. This also touches our topic, the water distribution networks. Let us have a global view on the development of water distribution in the least-developed countries regarding water access.

Therefore, we first import the data about about overall water access, provided by Gapminder Foundation and the World Bank. A look in the dataframe tells us that there is data of 168 countries worldwide, whereas not all the datasets are complete. We will use Plotly to visualize the development of the 10 countries having the worst access to water in 2000.

In [ ]:
#Using Pandas to read the csv file and sort the data
water = pd.read_csv('data/gapminder_water_access.csv', index_col=0)
ranking2000 = water.sort_values('2000')
worst2000 = ranking2000.head(10)

In [ ]:
#Using plotly for visualization:
fig = px.line(worst2000.T, color_discrete_sequence=px.colors.diverging.balance)

#add a title. The content of the following line could also be part of the brackets above:
fig.update_layout(title = 'Development of 10 countries with poorest water distribution in year 2000')

#costumize the axis
fig.update_xaxes(title='Year', ticklabelmode="period", dtick = 1)
fig.update_yaxes(title='overall water access [%]', dtick='10')

As you can see, eventhough the situation is not perfect yet, a lot of improvement has taken place!

Even if the lines have different colors, it might be difficult to distinguish between the different countries. The interactive graphic helps out in two different ways: Either, you can hover over the lines and you will see the country as well as the x & y parameters, or you can use the legend on the right side to deselect (one click) or to isolate (double click) one country. Give it a try!

### Task 1: show a time-series analysis

In this task, you have to combine the two examples from above: You should create a visualization of the precipitation measurement in Sagelva from 2018 to 2020.

- create a pandas dataframe from the csv file named `'Sagelva_Precipitation.csv'`from the folder `'data'` and have a look on it
- create a barplot using the plotly express library. Be aware of the following things:
  - Bars: The bars should not add up but stand beside each other --> use `barmode`
  - Title: Add a `title` for the plot
  - Axis: Add a titles and make sure that the dates are shown correctly on the x axes
  - Legend: make sure that you can select and deselect the different years
- Add the additional navigation tool `rangeslider`

In [ ]:
# Your code here...

[Solution to task 1](#Solution-to-task-1)

# 2. Statistical analysis: Histogram

Back to the first data set about the precipitation in Rotterdam.

A statistical analysis of the data can be helpful to find a pattern. The express library has therefore as well some tools inclded of which one will be shown here. On top, we use the `graph objects` library, usually imported as `go`, which is the basis for the plotly express library, in order to add a trace and show the daily precipitation as well.

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.histogram(raindailyRotterdam, x='YYYYMMDD', y='RH', histfunc='avg', title="Monthly Average Rainfall", range_y=['-1','25'], range_x=['2012-01-01', '2012-12-31'])
fig.update_traces(xbins_size="M1")
fig.update_xaxes(title='2012',showgrid=True, ticklabelmode="period", dtick="M1", tickformat="%b\n%Y")
fig.update_layout(bargap=0.05)
fig.add_trace(go.Scatter(mode='markers', x = raindailyRotterdam['YYYYMMDD'], y = raindailyRotterdam['RH'], name="daily"))

### Task 2: Time series analysis

Please use the rainfall data of Rotterdam to create a barplot of the total rainfall per month. Represent the daily variation with a line.

In [ ]:
# Your code here...

[Solution to task 2](#Solution-to-task-2)

# 3. 3D plots

Plotly enables you to create 3-dimensional graphs, that you can pan and zoom using the mouse.

### 3D scatter plots

The simplest form of these are scatter plots. These can be used to graph point that are dependend on 2 variables.

Further examples can be found in the official [documentation](https://plotly.com/python/3d-scatter-plots/).

Here is an example of the measured pressures at different scensors (hydrants) for different scenarios in a water distribution system. Used are the final results of the 2nd Assignment of the Hydroinformatics course.

The margins are removed, because axis labels can't move into the margin areas in 3d plots.

In [ ]:
# import packages
import plotly.express as px
import numpy as np
import pandas as pd

#read in data and convert into usable format
df = pd.read_csv('data/pressures.csv', decimal='.', sep=',')
df = df.melt('Sensors')
df.columns = ['Sensors', 'Scenarios', 'Pressure [m]']
df2 = pd.read_csv('data/SSE_values.csv', decimal='.', sep=',')
df2 = df2.melt('Sensors')
df.insert(3, 'SSE', df2["value"])

#plot the data
fig = px.scatter_3d(df, x='Sensors', y='Scenarios', z='Pressure [m]', height=700)

#remove margins
fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))

fig.show()

You can use your mouse to rotate the 3D plot and the mousewheel to zoom in. Try it out!

As a forth dimension color can be used. In the following plot the sum of squared errors (SSE) between the measured and simulated pressures over all scenarios and sensors in the water distribution system are indicated by their `color`.
Furthermore can the `opacity`be changed

In [ ]:
#plot the data
fig = px.scatter_3d(df, x="Sensors", y="Scenarios", z='Pressure [m]', color = "SSE", 
                    opacity = .6, title="Measured pressures", width=800, height=800)

fig.show()

To change the appearence of 3-dimenionsal plots the function `fig.update_scenes` is used. Here are a few examples of what can be changed. Also check the [documentation](https://plotly.com/python/reference/layout/scene/).

- `xaxis_nticks` changes the number of ticks on the xaxis.
- `camera_projection_type`can be used to change the camera projection between "perspective" and "orthographic".
- `xaxis_color` changes all colors associated with this axis.
- `xaxis_gridcolor`can be used to change the color of the x-axis grid. RGB-values can also be used to define the color.
- `xaxis_backgroundcolor`is used to change the color of the backround.
- `xaxis_tickangle`is used to change the angle of the ticklabels.
- `xaxis_ticks`is used to disable the tickmarks
- `xaxis_tickfont` is used to change the `color`, the `family`(the font) and the `size`of the tick labels.

To avoid the labels from getting cut off and make sure the title is displayed on the sides, the `margin` can be set with the function `fig.update_layout`. In 3d plots, axis labels don't actually pertrude into this margin area though.

In [ ]:
# change appearence
fig.update_scenes(xaxis_nticks=12, 
                  camera_projection_type="orthographic", 
                  xaxis_color="black",
                  xaxis_gridcolor='rgb(204, 204, 204)',
                  yaxis_gridcolor='rgb(204, 204, 204)',
                  zaxis_gridcolor='rgb(204, 204, 204)',
                  xaxis_backgroundcolor="white",
                  yaxis_backgroundcolor="white",
                  zaxis_backgroundcolor="white",
                  xaxis_tickangle= 30, 
                  xaxis_ticks="", yaxis_ticks="", 
                  xaxis_tickfont=dict(size=11, family="PT Sans Narrow"),
                  yaxis_tickfont=dict(size=11, family="PT Sans Narrow"),
                  zaxis_tickfont=dict(size=11, family="PT Sans Narrow"))

# increase margin
fig.update_layout(margin=dict(l=50, r=50, t=100, b=50))

### 3D surface plots

The Rastrigin function is a well-known function, whith a lot of local optima, but only one global optimum. It is defined as:

\begin{align}
f(\mathbf{x}) \ = \ a \cdot D + \sum_{i=1}^{D} \left(x_{i}^{2} - a \cdot \cos (2 \pi x_i) \right)
\end{align}

For more details check it out on [Wikipedia](https://de.wikipedia.org/wiki/Rastrigin-Funktion)!

To display it as a 3-dimensional surface plot, we use the `go.Figure`function, part of `plotly.graph_objects`.

In [ ]:
# define the rastrigin function for D dimensions
def rastrigin(xvector, a=15):
    
    D = len(xvector)
    value = D * a 
    for x in xvector:
        value += x ** 2 - a * np.cos(2 * np.pi * x)
    return value

In [ ]:
# calculate data for desired range
x = np.linspace(-7, 7, 250)
X, Y = np.meshgrid(x, x)
Z = rastrigin([X,Y])

In [ ]:
#create surface figure
import plotly.graph_objects as go

fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z)],)

fig.update_layout(title='Rastrigin function',
                  width=800, height=800,
                  margin=dict(l=65, r=50, b=90, t=90))

fig.show()

You can see the many local optima, but there is only one global minimum, which is located at (0,0).

### Task 3: Create a surface plot of a monkey saddle

Use your gained knowledge to create a 3d surface plot of a "monkey saddle". 
A monkey saddle is the name of the following function: 
\begin{align}
z \ = \ x^{3}-3xy^{2}
\end{align}

For more information check it out on [Wikipedia](https://en.wikipedia.org/wiki/Monkey_saddle)!

- Show the graph in a range from -10 to 10 (in x and y direction)
- Change the colors to make it look like "dark mode"

In [ ]:
# Your code here...

[Solution to task 3](#Solution-to-task-3)

# 4. Map plotting

The map feature can be useful for any kind of geographical information. For instance, data regarding water-distribution networks can be visualized as well as socio-geographical behaviors like the distribution of a population within a country.

For this task, we import a table with cities and small towns in Norway. The data is downloaded from [SimpleMaps.com](https://simplemaps.com/data/no-cities) and adapted to the unicode to be readable with pandas. More information about pandas can be found in [Mark Bakkers Notebook 8](https://nbviewer.org/github/mbakker7/exploratory_computing_with_python/blob/master/notebook8_pandas/py_exploratory_comp_8_sol.ipynb).

In [ ]:
#Installation of the Packages
import pandas as pd
import plotly.express as px

#read data
placesNo = pd.read_csv('data/places_norway.csv')

We can use the [Scatter Mapbox](https://plotly.com/python/mapbox-layers/) function which requires latitudinal and longitudinal coordinates and can show us the cities in a color code according to the population.

- `dataframe, lat='...', lon='...` This function needs to know where to find the coordiates in the dataframe. Coordinates are given in the Decimal Degree format with one latitude (y - direction: North/South) and one longitude(x - direction: East/West) value.
- `layout.mapbox.style` defines the lowest layer, also known as base map. 
  - It refers to an open library and is used by known services as STRAVA, lonely planet and national geographic. 
  - Default is a white background, so if you want to have the map in the background you have to set a style. 
    - Three commonly used are `'open-street-map'`, `'carto-positron'` and `'stamen-terrain'`. 
    - Further styles can be seen as well in the [documentation](https://plotly.com/python/mapbox-layers/).
  - It is possible to use satellite images provided by the Unitest States Geological Survey (USGS).
- `hover_name` defines the upper row which is shown when you move your mouse and hover about a point on the map.
- `hover_data` will be shown in the box as well
- `zoom` defines the initial zoom level. 1 shows the entire world, 10 is suitable to show points in a city.
- The layout is updated to adapt the outputsize and add a title.

In [ ]:
map = px.scatter_mapbox(placesNo, lat='lat', lon='lng', 
                        mapbox_style = 'carto-positron', 
                        hover_name = 'City',
                        hover_data = ["Population", "Status"],
                        color = 'Administration', 
                        color_discrete_sequence=px.colors.cyclical.IceFire,
                        size = 'Population',
                        zoom=3.1)
map.update_layout(title='Cities in Norway',
                  width=950, height=800)
map.show()

As well as in the time series analysis, we can use the mouse to navigate through the map. Thereby we can extract more knowledge out of the interactive map as  would be possible in a standard map. Give it a try and find out the population of Trondheim!


### Task 4:  Create a map

Let's use the gained knowledge and create a map of all the swedish cities:
- The csv- file you need is called `'places_sweden.csv'`
- Select a darker basemap as in the example and adapt the color scheme.
- Make sure that the city names, the authority type as well as the population is shown. You may have to take a look into the dataframe to see the correct labels.
- Does this map need another initial zoom- level?

In [ ]:
# Your code here...

[Solution to task 4](#Solution-to-task-4)

# 5. Plotting a water distribution network

The `wntr`package can be used as a link between python and an EPANET network model. More information about EPANET is given in the lectures as well as in the [documentation](https://epanet22.readthedocs.io/en/latest/index.html). 

In the `wntr`package there is an implementation of `plotly`, which can be used to create basic interactive plots of the water network model. [Documentation](https://wntr.readthedocs.io/en/stable/apidoc/wntr.graphics.network.html?highlight=plot_interactive) 

But this functionality is quite limited, as only `node_attribute`s can be displayed. The following example shows how to create this type of interactive html file using the `wntr`package.

In [ ]:
import wntr

In [ ]:
# get elevation data
wn = wntr.network.WaterNetworkModel('data/Exercise_Original.inp')
elevation = wn.query_node_attribute('elevation')

# create plot (creates an html file)
wntr.graphics.plot_interactive_network(wn, filename='data/Interactive_Model.html', auto_open=False, 
                            node_attribute=elevation, node_cmap = "magma", 
                            title= "Elevation of Network Model", 
                            figsize = (854,480))

To display an html file inside of jupyter notebooks, the `IPython.display` can be used. 

In [ ]:
# display html file in jupyter notebooks
from IPython.display import IFrame
IFrame(src='data/Interactive_Model.html', width=980, height=550)

## 6. Animations

To create a simple animaton in a scatter plot, `animation_frame`and `animation_group`can be used. 
- `animation_frame` defines over which data the animation takes place. We use the time in years.
- `animation_group`: rows with the same group are considered as the same object and therefore animated from one frame to the next. 

Check out the [documentation](https://plotly.com/python/animations/) for more details.

In [ ]:
# Animated scatter plot
import pandas as pd
import plotly.express as px

df_worlddata = px.data.gapminder()
fig = px.scatter(df_worlddata, x="gdpPercap", y="lifeExp", animation_frame="year", animation_group="country",
           size="pop", color="continent", hover_name="country",
           log_x=True, size_max=55, range_x=[100,100000], range_y=[25,90])
fig.update_xaxes(title='GDP per capita [PPP]')
fig.update_yaxes(title='Life Expectancy [years]')
fig.show()

You can make use of the interactive features again! By hovering over the circles you can see, which country they represent and all their associated data. Furthermore, you can use the legend on the right side to deselect (one click) or to isolate (double click) one continent, to make the number of countries less overwhelming.

Try it out yourself!

The same concept can also be applied to barplots using the `px_bar` function.

In [ ]:
# Animated bar plot
px.bar(df_worlddata, x="continent", y="pop", color="continent",
  animation_frame="year", animation_group="country", range_y=[0,4000000000])
fig.update_xaxes(title='Continent')
fig.update_yaxes(title='Population')

### Task 6: Animate population change

Use the gained knowledge to create a barplot animation for the relative population change of African countries since 1952. 
- Use the gapminder data as a basis
- Display the countries' absolute population as a color palette

In [ ]:
# Your code here...

[Solution to task 6](#Solution-to-task-6)

# 7. Saving figures

It might be useful for you to save your figures in order to use them somewhere else. For sure, you can simply make screenshots and add them to your text. But the interesting part of the plotly package is that it's interactive. Therefore, it has to be saved in a different way.

### html

The easiest way is to write a html file which you can for instance use on another homepage or add to an email:

In [ ]:
fig.write_html('data/fig.html')

The last figures saved under the variable fig is saved now in the location `data/fig.html` or the path that you have chosen. It's important that the folder already exists. Take care to not overwrite your files!

Now, we can have a look on the saved figure as well here in the notebook:

In [ ]:
# display html file in jupyter notebooks
from IPython.display import IFrame
IFrame(src='data/fig.html', width=980, height=550)

You can use this code as well to open any other html file from your computer.

### Dash

Since it can open a whole new chapter to write a web application which uses the plotly library, we recommend you to go to the documentation of [Dash](https://plotly.com/dash/) in case you want to use the data on another platform. As well, for all the plots shown above, there a ways to let it run on a server using Dash, it is described in each documentation chapter at the end of the page. It might be worth it, there are many possibilities and as mentioned before, this is used by big compainies like National Geographic, Lonely Planet, and so on.

That's it, we hope you have learned something :)

[Click here to go back to the start](#Introduction) 

<hr style="border:0.2px solid black"> </hr>
<hr style="border:0.2px solid black"> </hr>

# Task solutions

<hr style="border:0.2px solid black"> </hr>
<hr style="border:0.2px solid black"> </hr>

# Solution to task 1

In [ ]:
import plotly.express as px
import pandas as pd

#1. read data
raindaily = pd.read_csv('data/Sagelva_Precipitation.csv', parse_dates=['Date'], index_col=0)

#create barplot
fig = px.bar(raindaily, barmode='group')

#add a title
fig.update_layout(title = 'Daily Precipitation in Sagelva')

#costumize the axis
fig.update_xaxes(title='Year', ticklabelmode="period", rangeslider_visible=True)
fig.update_yaxes(title='Daily precipitation [mm]')

Continue with chapter 2: [Statistical analysis](#2.-Statistical-analysis:-Histogram)

# Solution to task 2

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

#prepare data
raindailyRotterdam = pd.read_csv('data/rotterdam_rainfall_2012.txt', sep=',', skiprows=9, parse_dates=['YYYYMMDD'],skipinitialspace=True)
raindailyRotterdam.iloc[:,2] = raindailyRotterdam.iloc[:,2] * 0.1
raindailyRotterdam.loc[raindailyRotterdam.RH<0.0, 'RH'] = 0.0

#make plot
fig = px.histogram(raindailyRotterdam, x='YYYYMMDD', y='RH', histfunc='sum', title="Monthly Total Rainfall", range_y=['-1','160'], range_x=['2012-01-01', '2012-12-31'])
fig.update_traces(xbins_size="M1")
fig.update_xaxes(title = '2012', showgrid=True, ticklabelmode="period", dtick="M1", tickformat="%b\n%Y")
fig.update_layout(bargap=0.05)
fig.add_trace(go.Scatter(mode='lines', x = raindailyRotterdam['YYYYMMDD'], y = raindailyRotterdam['RH'], name="daily"))

Continue with chapter 3: [3D plotting](#3.-3D-plots)

# Solution to task 3

In [ ]:
#define the Monkey saddle function
def monkey(x,y):
    z=x**3-3*x*y**2
    return z

In [ ]:
#create xyz data for this formula
import numpy as np
r = np.linspace(-10, 10, 250)
x,y = np.meshgrid(r, r)
z = monkey(x,y)

In [ ]:
#make graph
import plotly.graph_objects as go
fig = go.Figure(data=[go.Surface(x=x, y=y, z=z)],)

fig.update_layout(title='Monkey saddle function',
                  width=800, height=800,
                  margin=dict(l=65, r=50, b=90, t=90))

#dark mode
fig.update_scenes(xaxis_gridcolor='white',
                  yaxis_gridcolor='white',
                  zaxis_gridcolor='white',
                  xaxis_backgroundcolor="black",
                  yaxis_backgroundcolor="black",
                  zaxis_backgroundcolor="black")
fig.show()

Continue with chapter 4: [Map plotting](#4.-Map-plotting)

# Solution to task 4

In [ ]:
import pandas as pd
import plotly.express as px

placesSe = pd.read_csv('data/places_sweden.csv')
map = px.scatter_mapbox(placesSe, lat='lat', lon='lng', 
                        mapbox_style = 'carto-darkmatter', 
                        hover_name = 'City',
                        hover_data = ["Population", "Status"],
                        color = 'Administration', 
                        color_discrete_sequence=px.colors.cyclical.IceFire,
                        size = 'Population',
                        zoom=3.25)
map.update_layout(title='Cities in Sweden',
                  width=950, height=800)
map.show()

Continue with chapter 5: [Plotting a water distribution network](#5.-Plotting-a-water-distribution-network)

# Solution to task 6

In [ ]:
#import packages
import pandas as pd
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

#import data
df_worlddata = px.data.gapminder()
df_africa = df_worlddata.loc[df_worlddata['continent']=='Africa']

#calculate relative population
for i,line in df_africa.iterrows():
    if line["year"]==1952:
        basepop = line["pop"]
    df_africa.at[i,"rel_pop"]= line["pop"]/basepop

In [ ]:
#create barplot
fig = px.bar(df_africa, x="country", y="rel_pop", color="pop",
             animation_frame="year", animation_group="country", range_y=[0,8], 
             title= "Relative population change in African countries since 1952")

fig.update_layout(width=1000, height=800)
fig['layout']['updatemenus'][0]['pad']=dict(r= 10, t= 150)
fig['layout']['sliders'][0]['pad']=dict( t= 150,)

#update axes labels
fig.update_xaxes(title='Country')
fig.update_yaxes(title='Relative Population (compared to 1952)')

fig.update_scenes(xaxis_nticks=100)
fig.show()

Finish with chapter 7: [Saving figures](#7.-Saving-figures)